In [47]:
'''Dataloader for fused point features.'''
import sys
sys.path.append('/home/daniel/spatial_understanding/benchmarks/openscene/')
import copy
from glob import glob
from os.path import join
import torch
import numpy as np
import SharedArray as SA
from dataset.point_loader import Point3DLoader


In [48]:
# detection_mapping = {
#     0  'animal': 'animal',
#     1  'movable_object.barrier': 'barrier',
#     2  'vehicle.bicycle': 'bicycle',
#     3  'vehicle.bus.bendy': 'bus',
#     4  'vehicle.bus.rigid': 'bus',
#     5  'vehicle.car': 'car',
#     6  'vehicle.motorcycle': 'motorcycle',
#     7  'vehicle.construction': 'other_vehicle',
#     8  'vehicle.other': 'other_vehicle',
#     9  'human.pedestrian.adult': 'pedestrian',
#     10 'human.pedestrian.child': 'pedestrian',
#     11 'human.pedestrian.construction_worker': 'pedestrian',
#     12 'human.pedestrian.police_officer': 'pedestrian',
#     13 'movable_object.trafficcone': 'traffic_cone',
#     14 'static_object.traffic_sign': 'traffic_sign',
#     15 'vehicle.ego_trailer': 'trailer',
#     16 'vehicle.trailer': 'trailer',
#     17 'vehicle.truck': 'truck'
# }


TRUCKSCENES_LABELS_TO_IDX = {"animal": 0 ,
                            "human.pedestrian.adult": 1 ,
                            "human.pedestrian.child": 2 ,
                            "human.pedestrian.construction_worker": 3 ,
                            "human.pedestrian.personal_mobility": 4 ,
                            "human.pedestrian.police_officer": 5 ,
                            "human.pedestrian.stroller": 6 ,
                            "human.pedestrian.wheelchair": 7 ,
                            "movable_object.barrier": 8 ,
                            "movable_object.debris": 9 ,
                            "movable_object.pushable_pullable": 10,
                            "movable_object.trafficcone": 11,
                            "static_object.bicycle_rack": 12,
                            "static_object.traffic_sign": 13,
                            "vehicle.bicycle": 14,
                            "vehicle.bus.bendy": 15,
                            "vehicle.bus.rigid": 16,
                            "vehicle.car": 17,
                            "vehicle.construction": 18,
                            "vehicle.emergency.ambulance": 19,
                            "vehicle.emergency.police": 20,
                            "vehicle.motorcycle": 21,
                            "vehicle.trailer": 22,
                            "vehicle.truck": 23,
                            "vehicle.train": 24,
                            "vehicle.other": 25,
                            "vehicle.ego_trailer": 26,
                            "unlabeled": 27
}



TRUCKSCENES_CLASS_REMAP = 256*np.ones(28) # map from 28 classes to 16 classes
TRUCKSCENES_CLASS_REMAP[0] = 0 # animal
TRUCKSCENES_CLASS_REMAP[1] = 9 # pedestrian
TRUCKSCENES_CLASS_REMAP[2] = 9 # pedestrian
TRUCKSCENES_CLASS_REMAP[3] = 9 # pedestrian
TRUCKSCENES_CLASS_REMAP[4] = 9 # pedestrian
TRUCKSCENES_CLASS_REMAP[5] = 9 # pedestrian
TRUCKSCENES_CLASS_REMAP[6] = 9 # pedestrian
TRUCKSCENES_CLASS_REMAP[7] = 9 # pedestrian
TRUCKSCENES_CLASS_REMAP[8] = 1 # barrier
TRUCKSCENES_CLASS_REMAP[11] = 13 # traffic cone
TRUCKSCENES_CLASS_REMAP[13] = 14 # traffic sign
TRUCKSCENES_CLASS_REMAP[14] = 2 # bicycle
TRUCKSCENES_CLASS_REMAP[15] = 3 # bus
TRUCKSCENES_CLASS_REMAP[16] = 3 # bus
TRUCKSCENES_CLASS_REMAP[17] = 5 # car
TRUCKSCENES_CLASS_REMAP[18] = 7 # construction vehicle
TRUCKSCENES_CLASS_REMAP[21] = 6 # motorcycle
TRUCKSCENES_CLASS_REMAP[22] = 16 # trailer
TRUCKSCENES_CLASS_REMAP[23] = 17 # truck



In [52]:

class FusedFeatureLoader(Point3DLoader):
    '''Dataloader for fused point features.'''

    def __init__(self,
                 datapath_prefix, # data_root
                 datapath_prefix_feat, # data_root_2d_fused_feature
                 voxel_size=0.05,
                 split='train', aug=False, memcache_init=False,
                 identifier=7791, loop=1, eval_all=False,
                 input_color = False,
                 ):
        super().__init__(datapath_prefix=datapath_prefix, voxel_size=voxel_size,
                                           split=split, aug=aug, memcache_init=memcache_init,
                                           identifier=identifier, loop=loop,
                                           eval_all=eval_all, input_color=input_color)
        self.aug = aug
        self.input_color = input_color # decide whether we use point color values as input

        # prepare for 3D features
        self.datapath_feat = datapath_prefix_feat

        # Precompute the occurances for each scene
        # for training sets, ScanNet and Matterport has 5 each, nuscene 1
        # for evaluation/test sets, all has just one
        if 'nuscenes' in self.dataset_name or 'truckscenes' in self.dataset_name: # only one file for each scene
            self.list_occur = None
        
        if len(self.data_paths) == 0:
            raise Exception('0 file is loaded in the feature loader.')

    def __getitem__(self, index_long):

        index = index_long % len(self.data_paths)
        if self.use_shm:
            locs_in = SA.attach("shm://%s_%s_%06d_locs_%08d" % (
                self.dataset_name, self.split, self.identifier, index)).copy()
            feats_in = SA.attach("shm://%s_%s_%06d_feats_%08d" % (
                self.dataset_name, self.split, self.identifier, index)).copy()
            labels_in = SA.attach("shm://%s_%s_%06d_labels_%08d" % (
                self.dataset_name, self.split, self.identifier, index)).copy()
        else:
            locs_in, feats_in, labels_in = torch.load(self.data_paths[index])
            
            if 'truckscenes' in self.dataset_name:
                labels_in = [TRUCKSCENES_LABELS_TO_IDX[l] if l in TRUCKSCENES_LABELS_TO_IDX else 255 for l in labels_in]
                labels_in = np.array([TRUCKSCENES_CLASS_REMAP[l] for l in labels_in])
                locs_in = np.ascontiguousarray(locs_in[:, :3])
            else:
                labels_in[labels_in == -100] = 255

            labels_in = labels_in.astype(np.uint8)
            if np.isscalar(feats_in) and feats_in == 0:
                # no color in the input point cloud, e.g nuscenes lidar
                feats_in = np.zeros_like(locs_in)
            else:
                feats_in = (feats_in + 1.) * 127.5

        # load 3D features
        if self.dataset_name == 'scannet_3d':
            scene_name = self.data_paths[index][:-15].split('/')[-1]
        else:
            scene_name = self.data_paths[index][:-4].split('/')[-1]
            
        print(scene_name)

        if 'nuscenes' not in self.dataset_name and 'truckscenes' not in self.dataset_name:
            n_occur = self.list_occur[index]
            if n_occur > 1:
                nn_occur = np.random.randint(n_occur)
            elif n_occur == 1:
                nn_occur = 0
            else:
                raise NotImplementedError

            processed_data = torch.load(join(
                self.datapath_feat, scene_name+'_%d.pt'%(nn_occur)))
        else:
            # no repeated file
            processed_data = torch.load(join(self.datapath_feat, scene_name+'.pt'))

        flag_mask_merge = False
        if len(processed_data.keys())==2:
            flag_mask_merge = True
            feat_3d, mask_chunk = processed_data['feat'], processed_data['mask_full']
            if isinstance(mask_chunk, np.ndarray): # if the mask itself is a numpy array
                mask_chunk = torch.from_numpy(mask_chunk)
            mask = copy.deepcopy(mask_chunk)
            if self.split != 'train': # val or test set
                feat_3d_new = torch.zeros((locs_in.shape[0], feat_3d.shape[1]), dtype=feat_3d.dtype)
                feat_3d_new[mask] = feat_3d
                feat_3d = feat_3d_new
                mask_chunk = torch.ones_like(mask_chunk) # every point needs to be evaluted
        elif len(processed_data.keys())>2: # legacy, for old processed features
            feat_3d, mask_visible, mask_chunk = processed_data['feat'], processed_data['mask'], processed_data['mask_full']
            mask = torch.zeros(feat_3d.shape[0], dtype=torch.bool)
            mask[mask_visible] = True # mask out points without feature assigned

        if len(feat_3d.shape)>2:
            feat_3d = feat_3d[..., 0]

        locs = self.prevoxel_transforms(locs_in) if self.aug else locs_in

        # calculate the corresponding point features after voxelization
        if self.split == 'train' and flag_mask_merge:
            locs, feats, labels, inds_reconstruct, vox_ind = self.voxelizer.voxelize(
                locs_in, feats_in, labels_in, return_ind=True)
            vox_ind = torch.from_numpy(vox_ind)
            mask = mask_chunk[vox_ind] # voxelized visible mask for entire point cloud
            mask_ind = mask_chunk.nonzero(as_tuple=False)[:, 0]
            index1 = - torch.ones(mask_chunk.shape[0], dtype=int)
            index1[mask_ind] = mask_ind

            index1 = index1[vox_ind]
            chunk_ind = index1[index1!=-1]

            index2 = torch.zeros(mask_chunk.shape[0])
            index2[mask_ind] = 1
            index3 = torch.cumsum(index2, dim=0, dtype=int)
            # get the indices of corresponding masked point features after voxelization
            indices = index3[chunk_ind] - 1

            # get the corresponding features after voxelization
            feat_3d = feat_3d[indices]
        elif self.split == 'train' and not flag_mask_merge: # legacy, for old processed features
            feat_3d = feat_3d[mask] # get features for visible points
            locs, feats, labels, inds_reconstruct, vox_ind = self.voxelizer.voxelize(
                locs_in, feats_in, labels_in, return_ind=True)
            mask_chunk[mask_chunk.clone()] = mask
            vox_ind = torch.from_numpy(vox_ind)
            mask = mask_chunk[vox_ind] # voxelized visible mask for entire point clouds
            mask_ind = mask_chunk.nonzero(as_tuple=False)[:, 0]
            index1 = - torch.ones(mask_chunk.shape[0], dtype=int)
            index1[mask_ind] = mask_ind

            index1 = index1[vox_ind]
            chunk_ind = index1[index1!=-1]

            index2 = torch.zeros(mask_chunk.shape[0])
            index2[mask_ind] = 1
            index3 = torch.cumsum(index2, dim=0, dtype=int)
            # get the indices of corresponding masked point features after voxelization
            indices = index3[chunk_ind] - 1

            # get the corresponding features after voxelization
            feat_3d = feat_3d[indices]
        else:
            locs, feats, labels, inds_reconstruct, vox_ind = self.voxelizer.voxelize(
                locs[mask_chunk], feats_in[mask_chunk], labels_in[mask_chunk], return_ind=True)
            vox_ind = torch.from_numpy(vox_ind)
            feat_3d = feat_3d[vox_ind]
            mask = mask[vox_ind]

        if self.eval_all: # during evaluation, no voxelization for GT labels
            labels = labels_in
        if self.aug:
            locs, feats, labels = self.input_transforms(locs, feats, labels)
        coords = torch.from_numpy(locs).int()
        coords = torch.cat((torch.ones(coords.shape[0], 1, dtype=torch.int), coords), dim=1)
        if self.input_color:
            feats = torch.from_numpy(feats).float() / 127.5 - 1.
        else:
            # hack: directly use color=(1, 1, 1) for all points
            feats = torch.ones(coords.shape[0], 3)
        labels = torch.from_numpy(labels).long()

        if self.eval_all:
            return coords, feats, labels, feat_3d, mask, torch.from_numpy(inds_reconstruct).long()
        return coords, feats, labels, feat_3d, mask

def collation_fn(batch):
    '''
    :param batch:
    :return:    coords: N x 4 (batch,x,y,z)
                feats:  N x 3
                labels: N
                colors: B x C x H x W x V
                labels_2d:  B x H x W x V
                links:  N x 4 x V (B,H,W,mask)

    '''
    coords, feats, labels, feat_3d, mask_chunk = list(zip(*batch))

    for i in range(len(coords)):
        coords[i][:, 0] *= i

    return torch.cat(coords), torch.cat(feats), torch.cat(labels), \
        torch.cat(feat_3d), torch.cat(mask_chunk)


def collation_fn_eval_all(batch):
    '''
    :param batch:
    :return:    coords: N x 4 (x,y,z,batch)
                feats:  N x 3
                labels: N
                colors: B x C x H x W x V
                labels_2d:  B x H x W x V
                links:  N x 4 x V (B,H,W,mask)
                inds_recons:ON

    '''
    coords, feats, labels, feat_3d, mask, inds_recons = list(zip(*batch))
    inds_recons = list(inds_recons)

    accmulate_points_num = 0
    for i in range(len(coords)):
        coords[i][:, 0] *= i
        inds_recons[i] = accmulate_points_num + inds_recons[i]
        accmulate_points_num += coords[i].shape[0]

    return torch.cat(coords), torch.cat(feats), torch.cat(labels), \
        torch.cat(feat_3d), torch.cat(mask), torch.cat(inds_recons)

In [53]:
split = "val"
dataroot = "/home/daniel/spatial_understanding/benchmarks/openscene/data"

dataloader = FusedFeatureLoader(datapath_prefix=f"{dataroot}/truckscenes_3d",
                                datapath_prefix_feat=f"{dataroot}/truckscenes_multiview_openseg_{split}",
                                voxel_size=0.01, 
                                split=split, aug=False,
                                memcache_init=False, eval_all=True, identifier=6797,
                                input_color=False)

In [54]:
dataloader[0]

IndexError: index 255 is out of bounds for axis 0 with size 28